In [1]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# 获取物理gpu, cpu对象
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')

# 设置当前程序的　物理可见设备范围
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

#设置仅在需要时申请：
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# 设置在物理gpu上设置虚拟gpu，并用来限制gpu内存使用
tf.config.experimental.set_virtual_device_configuration(
    gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024), tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

# 获取当前虚拟gpu对象
logical_gpus = tf.config.experimental.list_logical_devices('GPU')

print('物理gpu个数: ', len(gpus))
print('总的gpu个数，其中包括括虚拟Logical gpus: ', len(logical_gpus))

物理gpu个数:  2
总的gpu个数，其中包括括虚拟Logical gpus:  2


In [3]:
# 获取数据
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 定义预处理函数
def data_scale(x, y):
    x = tf.cast(x, tf.float32)
    # 由于还没有学卷积，所以需要把图片数据展开,28*28
    x = x / 255
    x = tf.reshape(x,(784, -1))    # (78,1)
    x = tf.squeeze(x)    # 降一个维度
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_scale).cache().batch(64)
# next(iter(train_db))  # (shape=(64, 784))

In [4]:
# 创建模型
# 由于还没有学卷积，所以需要把图片数据展开，展开图像的长度为 28*28
num_inputs = 784
# 分类类别数为: 10 分类
num_outputs = 10
# 构建两个隐藏层
W1 = tf.Variable(tf.random.normal(shape=(num_inputs, 128), mean=0, stddev=0.1, dtype=tf.float32))
B1 = tf.Variable(tf.random.normal(shape=(1, 128), mean=0, stddev=0.1, dtype=tf.float32))

W2 = tf.Variable(tf.random.normal(shape=(128, 64), mean=0, stddev=0.1, dtype=tf.float32))
B2 = tf.Variable(tf.random.normal(shape=(1, 64), mean=0, stddev=0.1, dtype=tf.float32))

W3 = tf.Variable(tf.random.normal(shape=(128, 10), mean=0, stddev=0.1, dtype=tf.float32))
B3 = tf.Variable(tf.random.normal(shape=(1, 64), mean=0, stddev=0.1, dtype=tf.float32))
def hidden_layer_net(X):
    h1 = tf.nn.relu(tf.matmul(X, W1) + B1)
    h2 = tf.nn.relu(tf.matmul(h1, W2) + B2)
    out_put = tf.math.softmax(tf.matmul(h2, W3) + B3)
    return out_put